In [19]:
import numpy as np
import pandas as pd
import cv2, os

In [15]:
df = pd.read_csv('PCB_annotations_dataset_alt.csv', sep=';')


In [16]:
def generate_pixel_matrix(df, filename):
    """
    generates a black image of the same shape as the passed PCB image, with white white pixels 
    exactly inside the defect bounding box(es) of the passed PCB image 
    returns:
        the pixel matrix (numpy.ndarray)
        filename (string)

    Args:
        df (pandas.DataFrame): a PCB-annotation DataFrame 
        filename (string): the filename of the PCB image without file type ending
    """
    df_grouped = df.groupby('filename')
    # create a dataframe for each annotation file with as many rows as there are defects
    pcb = df_grouped.get_group(filename)
    # create a width x height marix of zeros, i.e. black pixels
    mask = np.zeros((pcb['width'].iloc[0], pcb['height'].iloc[0]))
    # for each defect set the pixels inside the retrieved bounding box to white
    for row in range(pcb.shape[0]):
        for i in range(pcb.ymin.iloc[row], pcb.ymax.iloc[row]+1):
            for j in range(pcb.xmin.iloc[row], pcb.xmax.iloc[row]+1):
                mask[i][j] = 255
    return(mask, filename)

In [20]:
# empty the subfolder 'Pixel_masks'
for filename in os.listdir('data/Pixel_masks'):
    os.remove(os.path.join(f'data/Pixel_masks/{filename}'))

# call the function generate_pixel_matrix once for each PCB in the annotation file
for filename in df.filename.unique():
    mask, filename = generate_pixel_matrix(df, filename)
    cv2.imwrite(f"data/Pixel_masks/pm-{filename}.png", mask)